In [1]:
import pandas as pd
import numpy as np
import pickle as p
import time

In [2]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=20, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

test_project_id = 'test_12'

### `/train`

In [26]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': False, 'project_id': test_project_id, 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [7]:
import requests
import json

# url = 'https://fillet.azurewebsites.net/query_progress/'
url = 'http://localhost:5000/query_progress/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print(r.json())

{'cv_done': 0, 'cv_progress': 0.0, 'ee_done': 1, 'fi_done': 1, 'pct_complete': 100.0, 'project_items': [202, 1230, 16019, 3732, 3766], 'train_complete': [202, 1230, 16019, 3732, 3766]}


### `/batch_query_progress`

In [47]:
import requests
import json

url = 'https://fillet.azurewebsites.net/batch_query_progress/'
# url = 'http://localhost:5000/batch_query_progress/'


payload = {'project_id_ls': ['8b59136f-5ca6-4a10-9e65-bb4171e887bd','c759e900-fbc7-4b59-82db-1fd8a81a0f2c']}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print(r.json())

{'8b59136f-5ca6-4a10-9e65-bb4171e887bd': {'cv_done': 1, 'cv_progress': 100.0, 'ee_done': 0, 'fi_done': 1, 'pct_complete': 100.0, 'project_items': [1, 2, 11, 12, 2571, 3601, 21, 22, 2581, 2588, 2589, 5662, 31, 5666, 5690, 1615, 6750, 101, 102, 4725, 4728, 1152, 1153, 16000, 16001, 3716, 3717, 3718, 16002, 16003, 3721, 3722, 3723, 5771, 2701, 3726, 3727, 3728, 16016, 16017, 3731, 3732, 3733, 16018, 16019, 3736, 3737, 3738, 3742, 3743, 2721, 4265, 1195, 2731, 4267, 4268, 4270, 2736, 4272, 4273, 1203, 1204, 1205, 3766, 3767, 3768, 7159, 3771, 3772, 3773, 5820, 196, 197, 1221, 200, 201, 202, 203, 204, 1225, 1230, 1231, 1232, 2256, 2257, 2258, 2259, 3816, 3817, 3818, 3821, 3822, 3823, 2313, 7434, 2315, 2316, 281, 284, 7965, 7966, 7967, 2867, 4409, 4410, 4411, 2887, 5469, 350, 7008, 7009, 7010, 4975, 4976, 4977, 4978, 5501, 7041, 7575, 7078, 7079, 7080, 5060, 2501, 7108, 7109, 7110, 7113, 7114, 7115, 5068, 5069, 5581, 2511, 5073, 5074, 7128, 2521, 7129, 1502, 1503, 1504, 2531, 3045, 3046, 304

### `/get_cv_results`

In [5]:
import requests
import json

# url = 'https://fillet.azurewebsites.net/get_cv_results/'
url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,3726.0,74.383606,0.698850,13.771327,0.185139,17.093644
1,2589.0,161.616043,0.697166,37.286499,0.230710,51.235978
2,3743.0,132.480972,0.759348,27.196514,0.205286,36.736907
3,202.0,2110.440186,0.641251,447.862620,0.212213,638.000844
4,16019.0,109.613182,0.639746,20.519545,0.187200,27.767439
5,1230.0,8.996674,0.351174,4.440118,0.493529,5.932912
6,3732.0,126.142509,0.707036,22.852803,0.181167,29.973956
7,2531.0,318.983582,0.664000,57.516393,0.180311,74.335335
8,3766.0,91.448227,0.691488,20.340310,0.222424,28.154570
9,1195.0,129.492813,0.658160,26.848710,0.207337,34.856421


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [4]:
import json
with open('constraints_3.json') as f:
    constraints = json.load(f)
    
import requests
import json
from threading import Thread

# url = 'https://fillet.azurewebsites.net/optimize/'
url = 'http://localhost:5000/optimize/'
    
payload = {'project_id': test_project_id,
           'optimisation_id':'manual_optimization_test_1', 
           'constraints': constraints, # two parts, first part constraint, second part price range limits
           'population': 300,
           'max_epoch': 10
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
# r = requests.post(url, data=payload, headers=headers, timeout=240)

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'headers': headers
       }).start()

In [5]:
a = [1, 'str', (1,2), None]

In [6]:
str(a)

"[1, 'str', (1, 2), None]"

### /get_opti_results

In [37]:

import requests
import json

url = 'https://fillet.azurewebsites.net/get_opti_results/'
# url = 'http://localhost:5000/get_opti_results/'


payload = {'project_id': test_project_id,
          'optimisation_id': 'manual_optimization_test_1'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=10)
r.json()






{'price_cols': ['Price_202',
  'Price_350',
  'Price_1195',
  'Price_1221',
  'Price_1230',
  'Price_2531',
  'Price_2589',
  'Price_2701',
  'Price_2867',
  'Price_3726',
  'Price_3732',
  'Price_3743',
  'Price_3766',
  'Price_5469',
  'Price_7008',
  'Price_7114',
  'Price_7157',
  'Price_7434',
  'Price_7575',
  'Price_16019'],
 'report': [29831.141954717732, 36696.16121196747, 0.0, 1.0],
 'report_info': ['estimated profit of the optimized price',
  'estimated revenue of the optimized price',
  'number of hard constraints (including price ranges) violated',
  'number of soft constraints (preferences) violated'],
 'result': [2.15,
  2.75,
  3.55,
  3.3,
  3.6,
  6.6,
  11.1,
  5.95,
  5.9,
  10.85,
  12.55,
  11.95,
  10.3,
  3.5,
  4.0,
  3.8,
  7.35,
  4.4,
  3.8,
  14.15],
 'success': True}

### `/predict/`

In [21]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

# print('Input Prices Quantities:')
# a_input

In [23]:
# Predict Sales Qty and Compare to Actual

import requests
import json

# url = 'https://fillet.azurewebsites.net/predict/'
url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': test_project_id, 'modeltype': 'default'}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_1195,214.0,331.0
Qty_1221,829.0,901.0
Qty_1230,35.0,34.0
Qty_16019,292.0,334.0
Qty_202,5171.0,5753.0
Qty_2531,591.0,796.0
Qty_2589,374.0,429.0
Qty_2701,1240.0,1268.0
Qty_2867,1341.0,1861.0
Qty_350,426.0,602.0


### `/get_feature_importances`

In [53]:
import requests
import json
import pandas as pd

url = 'https://fillet.azurewebsites.net/get_feature_importances/'
# url = 'http://localhost:5000/get_feature_importances/'

payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['202']).sort_values('importance', ascending=False)

,feature_name,importance
6,Price_3726,0.157354
3,Price_202,0.123959
4,Price_2531,0.117571
5,Price_2589,0.117251
1,Price_1230,0.104129
7,Price_3732,0.093999
9,Price_3766,0.093018
8,Price_3743,0.072451
0,Price_1195,0.060886
2,Price_16019,0.059381


In [3]:
import requests
import json
import pandas as pd

url = 'https://fillet.azurewebsites.net/get_elasticity_estimates/'
# url = 'http://localhost:5000/get_elasticity_estimates/'

payload = {'project_id': test_project_id}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['16019'])

,baseline_qty,decrease_pct_chng,decrease_qty,increase_pct_chng,increase_qty
Qty_1195,125.0,-0.016000,123.0,-0.072000,116.0
Qty_1230,9.0,-0.222222,7.0,-0.111111,8.0
Qty_16019,149.0,-0.281879,107.0,0.013423,151.0
Qty_202,2097.0,-0.018598,2058.0,0.010491,2119.0
Qty_2531,256.0,-0.031250,248.0,0.015625,260.0
Qty_2589,128.0,0.046875,134.0,0.023438,131.0
Qty_3726,61.0,0.032787,63.0,-0.016393,60.0
Qty_3732,121.0,-0.024793,118.0,-0.033058,117.0
Qty_3743,122.0,-0.081967,112.0,0.090164,133.0
Qty_3766,77.0,-0.025974,75.0,0.064935,82.0


In [4]:
r.json()

{'1195': {'baseline_qty': {'Qty_1195': 125.0,
   'Qty_1230': 9.0,
   'Qty_16019': 149.0,
   'Qty_202': 2097.0,
   'Qty_2531': 256.0,
   'Qty_2589': 128.0,
   'Qty_3726': 61.0,
   'Qty_3732': 121.0,
   'Qty_3743': 122.0,
   'Qty_3766': 77.0},
  'decrease_pct_chng': {'Qty_1195': 0.016,
   'Qty_1230': 0.0,
   'Qty_16019': -0.03355704697986577,
   'Qty_202': 0.004291845493562232,
   'Qty_2531': 0.06640625,
   'Qty_2589': 0.0,
   'Qty_3726': 0.06557377049180328,
   'Qty_3732': -0.14049586776859505,
   'Qty_3743': -0.03278688524590164,
   'Qty_3766': -0.2077922077922078},
  'decrease_qty': {'Qty_1195': 127.0,
   'Qty_1230': 9.0,
   'Qty_16019': 144.0,
   'Qty_202': 2106.0,
   'Qty_2531': 273.0,
   'Qty_2589': 128.0,
   'Qty_3726': 65.0,
   'Qty_3732': 104.0,
   'Qty_3743': 118.0,
   'Qty_3766': 61.0},
  'increase_pct_chng': {'Qty_1195': 0.056,
   'Qty_1230': 0.0,
   'Qty_16019': -0.06711409395973154,
   'Qty_202': -0.010491177873152123,
   'Qty_2531': 0.12109375,
   'Qty_2589': 0.015625,
   

### /detect_conflict


In [51]:
import requests
import json

url = 'https://fillet.azurewebsites.net/detect_conflict/'
# url = 'http://localhost:5000/detect_conflict/'
with open('constraints_2.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [52]:
r.json()

{'conflict': 'No conflict'}